In [1]:
from lithops import Storage,storage
from lithops.multiprocessing import Pool
from itertools import product
import lithops
import json
import io
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
stg = Storage()
data = stg.get_object('analysis.data', 'processeddata/sentimentsData.csv').decode()
data = io.StringIO(data)
df = pd.read_csv(data, sep=",")
print(df)

                         Id        Date  Retweet             location  \
0       1275180321332289536  2020-06-22     True  Northern California   
1       1222225270972780545  2020-01-28     True               Panama   
2       1231665892821524481  2020-02-23    False                  NaN   
3       1241317669619597312  2020-03-21     True                  NaN   
4       1240298080940879872  2020-03-18     True                  NaN   
...                     ...         ...      ...                  ...   
104413  1296143097924124672  2020-08-19    False  41.282988-95.987779   
104414  1381504926212296704  2021-04-12     True                Earth   
104415  1251307219154599936  2020-04-18     True        Washington DC   
104416  1290772296278433793  2020-08-04    False       Chattanooga TN   
104417  1336369493975642112  2020-12-08     True        Washington DC   

                                                     Text  Sentiment  
0       Wear your mask.Help slow the spread of #COVI

In [6]:
def average(date, df):
    mean = df.loc[df['Date'] == date]['Sentiment'].mean()
    return(mean)

In [14]:
print(average('2021-01-18', df))

-0.03796050420168067


In [15]:
#print(df.describe())
print(df)
print(df.sort_values(by=['Id','Date'], ascending=[False, False]))
'''daySent = df[['Sentiment']].groupby(df['Date'],axis=0).mean()["Sentiment"]
dates = df['Date'].unique()
#rint(daySent)
#daySent.plot(x="Name", y="Sentiment", kind="scatter")
d = pd.DataFrame({'x':dates, 'y':daySent})
d.plot('Date', 'Sentiment', kind='scatter')'''

                         Id        Date  Retweet             location  \
0       1275180321332289536  2020-06-22     True  Northern California   
1       1222225270972780545  2020-01-28     True               Panama   
2       1231665892821524481  2020-02-23    False                  NaN   
3       1241317669619597312  2020-03-21     True                  NaN   
4       1240298080940879872  2020-03-18     True                  NaN   
...                     ...         ...      ...                  ...   
104413  1296143097924124672  2020-08-19    False  41.282988-95.987779   
104414  1381504926212296704  2021-04-12     True                Earth   
104415  1251307219154599936  2020-04-18     True        Washington DC   
104416  1290772296278433793  2020-08-04    False       Chattanooga TN   
104417  1336369493975642112  2020-12-08     True        Washington DC   

                                                     Text  Sentiment  
0       Wear your mask.Help slow the spread of #COVI

'daySent = df[[\'Sentiment\']].groupby(df[\'Date\'],axis=0).mean()["Sentiment"]\ndates = df[\'Date\'].unique()\n#rint(daySent)\n#daySent.plot(x="Name", y="Sentiment", kind="scatter")\nd = pd.DataFrame({\'x\':dates, \'y\':daySent})\nd.plot(\'Date\', \'Sentiment\', kind=\'scatter\')'

In [4]:
cat_totals = df.groupby("Location")["Sentiment"].mean().sort_values()
#cat_totals.plot(kind="barh", fontsize=4)
print(cat_totals)

KeyError: 'Location'

In [30]:
import pandas as pd
import numpy as np
import sys
import nltk
import re
import csv 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import pandas as pd 

fileN = sys.argv[1]

data = pd.read_csv(fileN, sep='\t',encoding = 'utf8',lineterminator='\n', usecols = [2,3], names=['id_str','text'] ,low_memory=False, dtype=str)

data["id_str"]=data["id_str"].astype(str)
data["text"]=data["text"].astype(str)
  
# Preprocessing 
def remove_string_special_characters(s): 
    stripped = str(s) 
    # removes special characters with ' ' 
    #stripped = re.sub('[^a-zA-z\s]', '', str(s)) 
    #stripped = re.sub('_', '', stripped) 
      
    # Change any white space to one space 
    stripped = re.sub('\s+', ' ', stripped) 

    # Remove urls
    stripped = re.sub(r"http\S+",'', stripped)
      
    # Remove start and end white spaces 
    stripped = stripped.strip() 
    if stripped != '': 
            return stripped.lower() 

data["text"]= data["text"].apply(remove_string_special_characters)
data['text'].dropna(inplace=True)

# Stopword removal  
stop_words = set(stopwords.words('english')) 
spanishSW = set(stopwords.words('spanish'))
stop_words.update(spanishSW)
for i, line in enumerate(data["text"]): 
    data["text"][i] = ' '.join([x for x in nltk.word_tokenize(line) if ( x not in stop_words )]) 
# Getting ngrams  
vectorizer = CountVectorizer(ngram_range = (2,3)).fit(data["text"]) 

bag_of_words = vectorizer.transform(data["text"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

with open(fileN[:-4]+"-ngram.csv", "w") as f:
    w = csv.writer(f)
    w.writerows(words_freq)

FileNotFoundError: [Errno 2] No such file or directory: '--ip=127.0.0.1'

In [ ]:
import sys
import collections
import multiprocessing as mp
import csv

d = collections.defaultdict(list)

def split(l):
    return l.split(',')

pool = mp.Pool(processes=4)
for keys in pool.map(split, csv.reader("D:/URV/3r/2nQ/SD/practicas/SD_BigDataChallenge/datasets/englishTweetsReduced.csv")):
    d[keys[0]].append(keys[1:])
print(d)

In [5]:
in_file = open("D:/URV/3r/2nQ/SD/practicas/SD_BigDataChallenge/datasets/englishTweetsReduced.csv")
print(in_file.readlines())

<_io.TextIOWrapper name='D:/URV/3r/2nQ/SD/practicas/SD_BigDataChallenge/datasets/englishTweetsReduced.csv' mode='r' encoding='cp1252'>


In [45]:
from lithops.multiprocessing import Pool

def f(name):
    return 'hello '+str(name)

if __name__ == '__main__':
    with Pool() as pool:
        p = pool.map_async(f, ('bob',))
        
    print(p.get())
    

2021-05-28 13:36:39,837 [INFO] lithops.config -- Lithops v2.3.3
2021-05-28 13:36:39,854 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-gb
2021-05-28 13:36:39,856 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: us-south - Namespace: david.gaseni@estudiants.urv.cat_dev
2021-05-28 13:36:39,858 [INFO] lithops.executors -- Serverless Executor created with ID: 921c95-23
2021-05-28 13:36:39,862 [INFO] lithops.invokers -- ExecutorID 921c95-23 | JobID M000 - Selected Runtime: buzzerage/sd_fullpackages:latest - 256MB
2021-05-28 13:36:39,867 [INFO] lithops.job.job -- ExecutorID 921c95-23 | JobID M000 - Uploading function and data - Total: 695.0B
2021-05-28 13:36:40,326 [INFO] lithops.invokers -- ExecutorID 921c95-23 | JobID M000 - Starting function invocation: None() - Total: 1 activations
2021-05-28 13:36:40,336 [INFO] lithops.invokers -- ExecutorID 921c95-23 | JobID M000 - View execution logs at C:\Users\alexm\A

In [ ]:
from multiprocessing import Pool

try:
        in_file = open("D:/URV/3r/2nQ/SD/practicas/SD_BigDataChallenge/datasets/englishTweetsReduced.csv")
        text = in_file.readlines()
        with Pool() as pool:
            p = pool.map_async(len, text)
            #result = p.map(len,processes=process, text.split('\n'))

        print(p.get())
except Exception as error:
        # If an error occurred print the message to the screen
        print(error)

In [35]:
from multiprocessing import Pool

try:
        in_file = open("D:/URV/3r/2nQ/SD/practicas/SD_BigDataChallenge/datasets/englishTweetsReduced.csv")
        text = in_file.readlines()
        with Pool() as pool:
            p = pool.map(len, text)
            #result = p.map(len,processes=process, text.split('\n'))

        print(p)
except Exception as error:
        # If an error occurred print the message to the screen
        print(error)

2021-05-28 16:55:19,851 [INFO] lithops.config -- Lithops v2.3.3
2021-05-28 16:55:19,871 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-gb
2021-05-28 16:55:19,875 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: us-south - Namespace: david.gaseni@estudiants.urv.cat_dev
2021-05-28 16:55:19,877 [INFO] lithops.executors -- Serverless Executor created with ID: 01615f-33
2021-05-28 16:55:19,883 [INFO] lithops.invokers -- ExecutorID 01615f-33 | JobID M000 - Selected Runtime: buzzerage/sd_fullpackages:latest - 1024MB
2021-05-28 16:55:19,911 [INFO] lithops.job.job -- ExecutorID 01615f-33 | JobID M000 - Uploading function and data - Total: 4.4MiB
2021-05-28 16:55:23,005 [INFO] lithops.invokers -- ExecutorID 01615f-33 | JobID M000 - Starting function invocation: None() - Total: 10 activations
2021-05-28 16:55:23,049 [INFO] lithops.invokers -- ExecutorID 01615f-33 | JobID M000 - View execution logs at C:\Users\alexm

In [2]:
from lithops.multiprocessing import Pool
i = 1000
j = 0
text = {
    'lines' : []
}
str_var = []

def cont(dicc):
    count = 0
    for d in dicc['lines']:
        count += len(d)
    return count


try:
        in_file = open("D:/URV/3r/2nQ/SD/practicas/SD_BigDataChallenge/datasets/englishTweetsReduced.csv")
        '''text1 = in_file.readlines()[j:i]
        text2 = in_file.readlines()[j:i]
        text3 = in_file.readlines()[j:i]
        text4 = in_file.readlines()[j:i]
        text5 = in_file.readlines()[j:i]
        text6 = in_file.readlines()[j:i]
        text7 = in_file.readlines()[j:i]
        text8 = in_file.readlines()[j:i]
        text9 = in_file.readlines()[j:i]'''
        full_text = in_file.readlines()
        with Pool() as pool:
            for i in range(0, 100000, 10000):
                text['lines'] = full_text[j:i]
                str_var.append(text)
                j = i
            
            p = pool.map(cont, str_var)
            print(p)
            #result = p.map(len,processes=process, text.split('\n'))
except Exception as error:
        # If an error occurred print the message to the screen
        print(error)

2021-05-28 20:21:52,887 [INFO] lithops.config -- Lithops v2.3.3
2021-05-28 20:21:52,903 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-gb
2021-05-28 20:21:52,904 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: us-south - Namespace: david.gaseni@estudiants.urv.cat_dev
2021-05-28 20:21:52,906 [INFO] lithops.executors -- Serverless Executor created with ID: 7301d4-1
2021-05-28 20:21:52,908 [INFO] lithops.invokers -- ExecutorID 7301d4-1 | JobID M000 - Selected Runtime: buzzerage/sd_fullpackages:latest - 1024MB
2021-05-28 20:21:52,938 [INFO] lithops.job.job -- ExecutorID 7301d4-1 | JobID M000 - Uploading function and data - Total: 4.4MiB
2021-05-28 20:21:55,709 [INFO] lithops.invokers -- ExecutorID 7301d4-1 | JobID M000 - Starting function invocation: None() - Total: 10 activations
2021-05-28 20:21:55,720 [INFO] lithops.invokers -- ExecutorID 7301d4-1 | JobID M000 - View execution logs at C:\Users\alexm\AppD